In [177]:
from data_tools.api import *
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from joblib import Parallel,delayed
import datetime

DB_INFO = dict(host='192.168.1.234',
               user='winduser',
               password='1qaz@WSX',
               db='wind')

conn = pymysql.connect(**DB_INFO, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
date='20191128'

| 一级分类                                | 二级分类 | 简称                 | 因子构建方式                                                        | 财报数据类型 | 是否哑变量 |
|-------------------------------------|------|--------------------|---------------------------------------------------------------|--------|-------|
| 治理内控                                | 盈余质量 | risk\_操纵性应计利润      | 操纵性应计利润，应计利润剔除显著但不可操控变量的两个变量的影响:<br>营业收入变化和固定资产水平，剔除方式为分行业回归      | 年报     | \-    |
| 治理内控                                | 盈余质量 | risk\_真实盈余管理       | "销售操控\+生产操控\+酌量性费用操控<br>销售操控：经营性现金流不能被营业收入和营业收入变化所解释的部分，取负值<br>生产操控：生成成本不能被营业收入和营业收入变化所解释的部分<br>酌量性费用操控：销售和管理费用不能被营业收入所解释的部分，取负值"   | 年报   | \-                 |
| 治理内控                                | 盈余质量 | risk\_mscore       | M\-Score：综合考虑应收账款、毛利率、资产质量、营业收入、折旧率、销售管理费用、财务杠杆与应计利润          | 年报     | \-    |
| 治理内控                                | 盈余质量 | risk\_在建工程增速       | 过去三年在建工程对时间回归的斜率除以平均在建工程（半年频）                                 | 半年报/年报 | \-    |
| 治理内控                                | 盈余质量 | risk\_在建工程与现金流不匹配度 | 购建固定资产、无形资产和其他长期资产支付的现金对<br> 固定资产变化量 与 在建工程变化量 回归，取在建工程量的回归系数的相反数 | 年报     | \-    |
| 治理内控                                | 盈余质量 | risk\_应计利润         | （营业利润 \- 经营性净现金流）/上一期总资产                                      | 年报     | \-    |
| 治理内控                                | 盈余质量 | risk\_研发资本化        | 研发费用资本化/研发投入                                                  | 年报     | \-    |
| 治理内控                                | 盈余质量 | risk\_会计师事务所变更     | 近三年内是否更换过会计师事务所                                               | \-     | 哑变量   |
| 治理内控                                | 股权风险 | risk\_质押占比         | 质押股票数/总股本                                                     | \-     | \-    |
| 治理内控                                | 股权风险 | risk\_报告期质押冻结      | 质押\+冻结股票数（报告期）/总股本                                            | 最新     | \-    |
| 治理内控                                | 股权风险 | risk\_信托占比         | 最新定期报告前十大流通股东中信托持股占比                                          | 最新     | \-    |
| 治理内控                                | 股权风险 | risk\_信托数量         | 最新定期报告前十大流通股东中信托数量                                            | 最新     | \-    |
| 治理内控                                | 人事变动 | risk\_CFO离职        | 近3个月有无CFO离职，有记为1分，无记为0分                                       | \-     | 哑变量   |
| 治理内控                                | 人事变动 | risk\_高管离职         | 近3个月高管离职个数是否大于3（含3\)，有记为1分，无记为0分                              | \-     | 哑变量   |
| 治理内控                                | 企业内控 | risk\_财报披露调整       | 近1年内有无调整过财报披露日，或在规定时间内没有公布财报，有记1分，无记0分                        | \-     | 哑变量   |
| 治理内控                                | 企业内控 | risk\_违规行为         | 近1年内有无违规行为，有记为1分，无记为0分                                        | \-     | 哑变量   |
| 治理内控                                | 企业内控 | risk\_立案调查         | 近1年内有无被证监会立案调查，有记为1分，无记为0分                                    | \-     | 哑变量   |
| 治理内控                                | 企业内控 | risk\_诉讼仲裁         | 近1年内诉讼案件个数是否大于3                                               | \-     | 哑变量   |
| 治理内控                                | 企业内控 | risk\_业绩预告不准确度     | 近4个财报期内净利润是否均高于业绩预告下限，不是为1分，是为0分                              | \-     | 哑变量   |
| 治理内控                                | 过度投资 | risk\_商誉           | 商誉/净资产绝对值                                                     | 最新     | \-    |
| 治理内控                                | 过度投资 | risk\_并购次数         | 3年内并购次数                                                       | \-     | \-    |


In [178]:
from data_tools.api import trade_days

trade_dates_all = trade_days.copy()
def get_prev_n_trade_date(trade_date, n):
    pos = np.searchsorted(trade_dates_all, trade_date)
    assert pos >= n
    return int(trade_dates_all[pos - n])

def get_next_n_trade_date(trade_date, n=1):
    pos = np.searchsorted(trade_dates_all, trade_date, side='right')
    assert pos + n - 1 < len(trade_dates_all)
    return int(trade_dates_all[pos + n - 1])


#公司代码和股票代码对照表
sql="SELECT S_INFO_WINDCODE,S_INFO_COMPCODE from WINDCUSTOMCODE where S_INFO_SECURITIESTYPES='A'"
df_codemap=pd.read_sql_query(sql,conn)

## 应计利润、操纵性应计利润

In [179]:
def get_accural_info(date,conn):
    sql_1="""select S_INFO_WINDCODE,REPORT_PERIOD,TOT_ASSETS,FIX_ASSETS from ASHAREBALANCESHEET 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-3)

    sql_2="""select S_INFO_WINDCODE,REPORT_PERIOD,OPER_PROFIT,OPER_REV from ASHAREINCOME 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-3)

    sql_3="""select S_INFO_WINDCODE,REPORT_PERIOD,NET_CASH_FLOWS_OPER_ACT from ASHARECASHFLOW 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-2)


    df_asset=pd.read_sql_query(sql_1,conn)
    df_income_sheet=pd.read_sql_query(sql_2,conn)
    df_net_oper_cashflow=pd.read_sql_query(sql_3,conn)

    ###上期的总资产以及本期的固定资产
    df_asset=df_asset.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])
    df_tot_asset_ly=df_asset.groupby("S_INFO_WINDCODE").apply(lambda x:x["TOT_ASSETS"].iloc[-2] if len(x)>1 else np.nan)\
                        .to_frame("TOT_ASSETS").reset_index()
    df_fixed_asset_ty=df_asset.groupby("S_INFO_WINDCODE").apply(lambda x:x["FIX_ASSETS"].iloc[-1])\
                        .to_frame("FIX_ASSETS").reset_index()
    
    ###营业利润与营业收入变化
    df_income_sheet=df_income_sheet.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])
    df_oper_rev_delta=df_income_sheet.groupby("S_INFO_WINDCODE")\
                                    .apply(lambda x:x["OPER_REV"].iloc[-1]-x["OPER_REV"].iloc[-2] if len(x)>1 else np.nan)\
                                    .to_frame("oper_rev_delta").reset_index()
    df_oper_profit_ty=df_income_sheet.groupby("S_INFO_WINDCODE").apply(lambda x:x.iloc[-1])
    
    ###经营性现金流净额
    df_net_oper_cashflow=df_net_oper_cashflow.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])
    df_net_oper_cashflow_ty=df_net_oper_cashflow.groupby("S_INFO_WINDCODE").apply(lambda x:x.iloc[-1])
    
    ###合并数据
    df_summary=df_tot_asset_ly.merge(df_fixed_asset_ty,on="S_INFO_WINDCODE")\
                            .merge(df_oper_profit_ty,on="S_INFO_WINDCODE")\
                            .merge(df_oper_rev_delta,on="S_INFO_WINDCODE")\
                            .merge(df_net_oper_cashflow_ty,on="S_INFO_WINDCODE")
    
    #应该可以不用除以总资产
    df_summary["accural_to_asset_ratio"]=(df_summary["OPER_PROFIT"]-df_summary["NET_CASH_FLOWS_OPER_ACT"])/df_summary["TOT_ASSETS"]
    df_summary["delta_operrev_to_asset_ratio"]=df_summary["oper_rev_delta"]/df_summary["TOT_ASSETS"]
    df_summary["fixed_asset_to_asset_ratio"] =df_summary["FIX_ASSETS"]/df_summary["TOT_ASSETS"]

    ###分行业回归
    def regress_by_indus(df):
        df=df[pd.notnull(df["fixed_asset_to_asset_ratio"])&pd.notnull(df["delta_operrev_to_asset_ratio"])&pd.notnull(df["accural_to_asset_ratio"])]
        factor=np.array(df[["delta_operrev_to_asset_ratio","fixed_asset_to_asset_ratio"]])
        accural=np.array(df["accural_to_asset_ratio"])

        factor=sm.add_constant(factor)
        result=sm.OLS(accural,factor).fit()
        return pd.Series(result.resid,index=df["S_INFO_WINDCODE"]).to_frame("accural_abnormal")

    df_indus=query_table("DailyBar",start_date=date,end_date=date,fields=["L1_INDUSTRY"])\
                        .drop("DataDate",axis=1).rename(columns={"sid":"S_INFO_WINDCODE"})
    df_summary=df_summary.merge(df_indus,on="S_INFO_WINDCODE")
    df_accural_abnormal=df_summary.groupby("L1_INDUSTRY").apply(regress_by_indus)
    df_accural_to_asset_ratio=df_summary[["accural_to_asset_ratio","S_INFO_WINDCODE"]]
    
    return df_accural_to_asset_ratio,df_accural_abnormal
get_accural_info(date,conn)

(      accural_to_asset_ratio S_INFO_WINDCODE
 0                   0.027591       000001.SZ
 1                   0.029073       000002.SZ
 2                   0.109541       000004.SZ
 3                   0.091872       000005.SZ
 4                  -0.041049       000006.SZ
 5                  -0.103401       000007.SZ
 6                   0.095857       000008.SZ
 7                  -0.060875       000009.SZ
 8                  -0.153788       000010.SZ
 9                  -0.064513       000011.SZ
 10                 -0.081807       000012.SZ
 11                  0.089798       000014.SZ
 12                  0.142795       000016.SZ
 13                  0.097539       000017.SZ
 14                 -0.195189       000018.SZ
 15                  0.039347       000019.SZ
 16                  0.041191       000020.SZ
 17                  0.016783       000021.SZ
 18                 -0.030674       000023.SZ
 19                  0.070453       000025.SZ
 20                 -0.028277     

## 销售操作、费用操作、生产操作

In [180]:
def get_profit_manipulate_info(date,conn):
    sql_1="""select S_INFO_WINDCODE,REPORT_PERIOD,TOT_ASSETS from ASHAREBALANCESHEET 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-3)

    sql_2="""select S_INFO_WINDCODE,REPORT_PERIOD,OPER_REV,LESS_SELLING_DIST_EXP,LESS_GERL_ADMIN_EXP,LESS_OPER_COST from ASHAREINCOME 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-3)

    sql_3="""select S_INFO_WINDCODE,REPORT_PERIOD,STOT_CASH_INFLOWS_OPER_ACT from ASHARECASHFLOW 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-2)


    df_tot_asset=pd.read_sql_query(sql_1,conn)
    df_income_sheet=pd.read_sql_query(sql_2,conn)
    df_oper_cashflow=pd.read_sql_query(sql_3,conn)

    ###上期总资产
    df_tot_asset=df_tot_asset.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])
    df_tot_asset_ly=df_asset.groupby("S_INFO_WINDCODE").apply(lambda x:x["TOT_ASSETS"].iloc[-2] if len(x)>1 else np.nan)\
                            .to_frame("TOT_ASSETS").reset_index()

    ###营业收入变化，营业成本，销售管理费用
    df_income_sheet=df_income_sheet.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])
    df_income_sheet["selling_admin_exp"]=df_income_sheet["LESS_SELLING_DIST_EXP"]+df_income_sheet["LESS_GERL_ADMIN_EXP"]
    df_oper_rev_delta=df_income_sheet.groupby("S_INFO_WINDCODE")\
                                     .apply(lambda x:x["OPER_REV"].iloc[-1]-x["OPER_REV"].iloc[-2] if len(x)>1 else np.nan)\
                                     .to_frame("oper_rev_delta").reset_index()
    df_oper_cost_ty=df_income_sheet.groupby("S_INFO_WINDCODE").apply(lambda x:x.iloc[-1])
    
    ###经营性现金流入
    df_oper_cashflow=df_oper_cashflow.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])
    df_oper_cashflow_ty=df_oper_cashflow.groupby("S_INFO_WINDCODE").apply(lambda x:x.iloc[-1])
    
    ###合并数据
    df_summary=df_tot_asset_ly.merge(df_oper_rev_delta,on="S_INFO_WINDCODE")\
                              .merge(df_oper_cost_ty,on="S_INFO_WINDCODE")\
                              .merge(df_oper_cashflow_ty,on="S_INFO_WINDCODE")
    
    #这一步应该可以省略
    for factor in ["LESS_OPER_COST","selling_admin_exp","STOT_CASH_INFLOWS_OPER_ACT","OPER_REV","oper_rev_delta"]:
        df_summary[factor]=df_summary[factor]/df_summary["TOT_ASSETS"]

    ###分行业回归
    def regress_by_indus(df):
        result_df=pd.DataFrame(index=df["S_INFO_WINDCODE"])
        df=df.dropna(axis=0)

        for factor in ["LESS_OPER_COST","selling_admin_exp","STOT_CASH_INFLOWS_OPER_ACT"]:
            try:
                explain_var=np.array(df[["OPER_REV","oper_rev_delta"]])
                factor_data=np.array(df[factor])

                explain_var=sm.add_constant(explain_var)
                result=sm.OLS(factor_data,explain_var).fit()
                result_df[factor+"_abnormal"]=pd.Series(result.resid,index=df["S_INFO_WINDCODE"]).to_frame(factor+"_abnormal")
            except ValueError:
                pass
        return result_df

    df_indus=query_table("DailyBar",start_date=date,end_date=date,fields=["L1_INDUSTRY"])\
                        .drop("DataDate",axis=1).rename(columns={"sid":"S_INFO_WINDCODE"})
    df_summary=df_summary.merge(df_indus,on="S_INFO_WINDCODE")
    df_profit_manipulate=df_summary.groupby("L1_INDUSTRY").apply(regress_by_indus)
    
    return df_profit_manipulate.reset_index()
get_profit_manipulate_info(date,conn)

,L1_INDUSTRY,S_INFO_WINDCODE,LESS_OPER_COST_abnormal,STOT_CASH_INFLOWS_OPER_ACT_abnormal,selling_admin_exp_abnormal
0,0,002963.SZ,0.082489,-0.402498,-0.062798
1,0,002965.SZ,0.076224,0.005923,-0.038545
2,0,002967.SZ,0.013855,-0.053739,0.039645
3,0,300564.SZ,0.062309,0.083977,-0.009453
4,0,300793.SZ,0.155540,0.201592,-0.083322
5,0,300795.SZ,-0.262849,0.003685,0.223141
6,0,300796.SZ,0.082427,-0.077494,-0.072550
7,0,300797.SZ,0.028387,0.079773,0.049605
8,0,300798.SZ,0.175409,-0.528407,-0.087901
9,0,300799.SZ,-0.135258,0.001547,-0.076004


## 在建工程增速

In [181]:
def get_const_in_growth(date,conn):
    sql_1="""select S_INFO_WINDCODE,REPORT_PERIOD,CONST_IN_PROG from ASHAREBALANCESHEET 
         where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and ANN_DT>='{}' and (REPORT_PERIOD like '%1231' or REPORT_PERIOD like '%0630')"""\
        .format(get_next_trade_date(date),get_prev_n_trade_date(date,500))
    df_const_in_prog=pd.read_sql_query(sql_1,conn)
    
    def calcu_growth(df):
        try:
            df=df.iloc[-3:]

            const_in_prog=np.array(df["CONST_IN_PROG"])
            const_in_prog=const_in_prog-const_in_prog.mean()
            year=np.array([1,2,3])
            year=year-year.mean()

            beta=np.sum(const_in_prog*year)/np.sum(year*year)
            return beta/np.array(df["CONST_IN_PROG"]).mean()
        except ValueError:
            return np.nan

    df_const_in_prog=df_const_in_prog.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])
    df_const_in_growth=df_const_in_prog.groupby("S_INFO_WINDCODE").apply(calcu_growth)
    return df_const_in_growth
get_const_in_growth(date,conn)

S_INFO_WINDCODE
000001.SZ         NaN
000002.SZ    0.689796
000003.SZ         NaN
000004.SZ         NaN
000005.SZ   -1.406551
000006.SZ         NaN
000007.SZ         NaN
000008.SZ    0.041147
000009.SZ    0.208315
000010.SZ         NaN
000011.SZ         NaN
000012.SZ    0.197685
000014.SZ         NaN
000015.SZ         NaN
000016.SZ    0.785066
000017.SZ         NaN
000018.SZ    0.729641
000019.SZ    1.022109
000020.SZ    0.661078
000021.SZ    0.250039
000023.SZ    0.037535
000025.SZ   -1.293867
000026.SZ    0.014874
000027.SZ   -0.121425
000028.SZ    0.017763
000029.SZ         NaN
000030.SZ   -0.078389
000031.SZ         NaN
000032.SZ    0.000000
000034.SZ    0.575641
               ...   
A19422.SH    0.860157
A19423.SH   -0.855537
A19424.SZ         NaN
A19425.SZ    0.260891
A19426.SZ         NaN
A19427.SH         NaN
A19428.SH   -0.313088
A19429.SH         NaN
A19430.SZ    0.621743
A19431.SH   -1.427554
A19432.SH         NaN
A19433.SH   -0.344456
A19434.SH   -1.241903
A19435.SH   -0.4

## 在建工程与现金流不匹配度
#### 因为觉得使用残差比excel 里面给出的算法更加科学，所以就改用残差作为因子

In [182]:
def cashpay_const_deviate(date,conn):
    sql_1="""select S_INFO_WINDCODE,REPORT_PERIOD,CONST_IN_PROG,FIX_ASSETS from ASHAREBALANCESHEET 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-3)

    sql_2="""select S_INFO_WINDCODE,REPORT_PERIOD,CASH_PAY_ACQ_CONST_FIOLTA from ASHARECASHFLOW 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-2)
    
    ###获得固定资产、在建工程数据以及构建此类资产支付现金流数据
    df_fixed_constructing_asset=pd.read_sql_query(sql_1,conn)
    df_cashpay_for_fixedasset=pd.read_sql_query(sql_2,conn)


    df_fixed_constructing_asset=df_fixed_constructing_asset.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])
    df_fixed_constructing_delta=df_fixed_constructing_asset.groupby("S_INFO_WINDCODE").\
                                apply(lambda x:x.iloc[-1,2:]-x.iloc[-2,2:] if len(x)>=2 else x.iloc[-1,2:]).\
                                reset_index()

    df_cashpay_for_fixedasset=df_cashpay_for_fixedasset.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])
    df_cashpay_for_fixedasset=df_cashpay_for_fixedasset.groupby("S_INFO_WINDCODE").apply(lambda x:x.iloc[-1])

    ###分行业回归
    def regress_by_indus(df):
        df=df.dropna()
        cashpay=np.array(df["CASH_PAY_ACQ_CONST_FIOLTA"])
        fixed_constructing_delta=np.array(df[["CONST_IN_PROG","FIX_ASSETS"]])
        fixed_constructing_delta=sm.add_constant(fixed_constructing_delta)
        result=sm.OLS(cashpay,fixed_constructing_delta).fit()
        return pd.Series(result.resid,index=df["S_INFO_WINDCODE"]).to_frame("cashpay_fixedconsting_abnormal")

    df_summary=df_fixed_constructing_delta.merge(df_cashpay_for_fixedasset,on=["S_INFO_WINDCODE"])
    df_indus=query_table("DailyBar",start_date=date,end_date=date,fields=["L1_INDUSTRY"])\
                        .drop("DataDate",axis=1).rename(columns={"sid":"S_INFO_WINDCODE"})
    df_summary=df_summary.merge(df_indus,on="S_INFO_WINDCODE")
    df_cashpay_const_deviate=df_summary.groupby("L1_INDUSTRY").apply(regress_by_indus)
    
    return df_cashpay_const_deviate.reset_index()
cashpay_const_deviate(date,conn)

,L1_INDUSTRY,S_INFO_WINDCODE,cashpay_fixedconsting_abnormal
0,0,002965.SZ,1.711493e+07
1,0,002967.SZ,1.671898e+08
2,0,300796.SZ,-5.093051e+07
3,0,300797.SZ,1.249919e+07
4,0,300798.SZ,-9.235639e+07
5,0,300800.SZ,-2.327721e+07
6,0,300801.SZ,5.478691e+07
7,0,300802.SZ,1.734950e+07
8,0,603390.SH,-1.679724e+07
9,0,603610.SH,-2.130697e+07


## 研发资本化

In [183]:
def get_RD_capitalize_ratio(date,conn):  
    ###研发资本化支出
    sql_1="""select S_INFO_COMPCODE,REPORT_PERIOD,ITEM_DATA,ANN_ITEM,ITEM_AMOUNT
            from ASHARERDEXPENDITURE where ANN_DT<={} and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-2)

    df=pd.read_sql_query(sql_1,conn)
    df=df.merge(df_codemap,on="S_INFO_COMPCODE").sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])
    def calcu_capitalize_expense(df):
        latest_repper=df["REPORT_PERIOD"][df.index[-1]]
        df=df[df["REPORT_PERIOD"]==latest_repper]

        capitalize_item=['研发投入资本化的金额(元)',
                         '本期资本化研发投入',
                         '研发支出资本化的金额(元)',
                         '本期资本化研发支出',
                         '研发投入资本化的金额',
                         '研发投入资本化的金额(万元)',
                         '研发投入资本化的金额(人民币元)',
                         '资本化研发支出占当期净利润的比重',
                         '2018年资本化研发投入',
                         '本报告期资本化研发支出',
                         '研发投入资本化的金额(人民币千元)',
                         '研发投入资本化的金额(千元)']
        df_capitalize_expense=df[df["ANN_ITEM"].isin(capitalize_item)]
        capitalize_expense=np.sum(df_capitalize_expense["ITEM_AMOUNT"])
        return pd.Series([latest_repper,capitalize_expense],index=["REPORT_PERIOD","capitalize_expense"])
    df_RD_capitalize_expense=df.groupby("S_INFO_WINDCODE").apply(calcu_capitalize_expense)

    #研发总支出
    sql_2="""select S_INFO_WINDCODE,REPORT_PERIOD,RD_EXPENSE from ASHAREFINANCIALINDICATOR 
        where ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-2)
    
    df_RD_total_expense=pd.read_sql_query(sql_2,conn)
    df_RD_total_expense=df_RD_total_expense.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])
    df_RD_total_expense=df_RD_total_expense.groupby("S_INFO_WINDCODE").apply(lambda x:x[["RD_EXPENSE","REPORT_PERIOD"]].iloc[-1])


    df_RD_capitalize_ratio=df_RD_capitalize_expense["capitalize_expense"]/df_RD_total_expense["RD_EXPENSE"]
    
    return df_RD_capitalize_ratio

df=get_RD_capitalize_ratio(date,conn)
df.isnull().sum()

1350

## 会计师事务所变更
#### 没有找到合适的数据库，只能在重大事件里面找了，但是有些不太科学，应该还要再做一下判别到底是不是会计师事务所变更

In [184]:
def get_accounter_change_info(date,conn):
    sql="""select S_INFO_WINDCODE,S_EVENT_ANNCEDATE,S_EVENT_CONTENT,S_EVENT_HAPDATE,S_EVENT_CATEGORYCODE 
            from ASHAREMAJOREVENT where S_EVENT_ANNCEDATE>='{}'""".format(get_prev_n_trade_date(date,750))
    df_major_event=pd.read_sql(sql,conn)
    def fun(x):
        try:
            #if '会计师事务所' in x and '变更' in x:
            #    print(x)
            return '会计师事务所' in x and '变更' in x
        except TypeError:
            return False
    df_major_event["is_accounter_change"]=df_major_event["S_EVENT_CONTENT"].apply(fun)
    df_accounter_change=df_major_event.groupby("S_INFO_WINDCODE").apply(lambda x:x["is_accounter_change"].sum()>0)
    
    return df_accounter_change

df=get_accounter_change_info(date,conn)
df.sum()

198

## 股票质押占比、质押+冻结（报告期）
#### 股票质押占比有另一个现成的数据，但是数据好像不是很合适

In [185]:
def get_stock_plefro_info(date,conn):
    ###股票质押占比
    sql="""select S_INFO_WINDCODE,ANN_DT,S_PLEDGE_BGDATE,S_PLEDGE_ENDDATE,S_PLEDGE_SHR_RATIO 
            from ASHAREEQUITYPLEDGEINFO where S_PLEDGE_ENDDATE>='{0}' and S_PLEDGE_BGDATE<='{0}'""".format(date)

    df_stock_pledge=pd.read_sql_query(sql,conn)
    df_stock_pledge_ratio=df_stock_pledge.groupby("S_INFO_WINDCODE").apply(lambda x:(np.sum(x["S_PLEDGE_SHR_RATIO"])))
    
    ###质押+冻结（报告期）
    sql = ("""select S_INFO_WINDCODE, TRADE_DT, TOT_SHR_TODAY from ASHAREEODDERIVATIVEINDICATOR"""
           + """ where TRADE_DT='%s'""" % date)
    df_total_share=pd.read_sql_query(sql,conn)
    
    sql="""select S_INFO_WINDCODE,PRICE_DATE,F_NAV_UNIT,F_NAV_ADJFACTOR
            from AEQUFROPLEINFOREPPEREND where F_NAV_UNIT<='{}' and PRICE_DATE>='{}'"""\
            .format(get_next_trade_date(date),get_prev_n_trade_date(date,150))

    df_report_plefro=pd.read_sql_query(sql,conn)
    df_report_plefro=df_report_plefro.sort_values(["S_INFO_WINDCODE","PRICE_DATE"])\
                                      .groupby("S_INFO_WINDCODE").apply(lambda x:x.iloc[-1])

    df_report_plefro_share=df_report_plefro.groupby("S_INFO_WINDCODE").apply(lambda x:np.sum(x["F_NAV_ADJFACTOR"])).to_frame("F_NAV_ADJFACTOR").reset_index()
    df_report_plefro_share=df_report_plefro_share.merge(df_total_share,on='S_INFO_WINDCODE')
    df_report_plefro_ratio=df_report_plefro_share['F_NAV_ADJFACTOR']/df_report_plefro_share['TOT_SHR_TODAY']/10000
    
    return df_stock_pledge_ratio,df_report_plefro_ratio
get_stock_plefro_info(date,conn)

/home/ywang/env/dev/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: 'S_INFO_WINDCODE' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version


(S_INFO_WINDCODE
 000002.SZ     0.4200
 000004.SZ    18.1200
 000005.SZ     3.1200
 000008.SZ     0.0000
 000018.SZ     5.1500
 000034.SZ     1.4400
 000038.SZ     0.0000
 000040.SZ     0.0000
 000042.SZ     0.0000
 000046.SZ     0.0000
 000060.SZ     1.4880
 000100.SZ     0.5000
 000158.SZ     5.0900
 000159.SZ     0.0000
 000407.SZ     5.6500
 000408.SZ     0.0000
 000413.SZ     2.6800
 000419.SZ     0.0000
 000428.SZ     0.0000
 000488.SZ     0.0000
 000498.SZ     2.7400
 000503.SZ     0.0000
 000506.SZ    25.0800
 000509.SZ     2.4200
 000510.SZ     7.3600
 000516.SZ     0.0000
 000518.SZ     0.0000
 000525.SZ     0.0000
 000538.SZ     0.0000
 000540.SZ     2.9200
               ...   
 603879.SH    40.6540
 603880.SH    28.5580
 603882.SH     5.0500
 603883.SH     7.6500
 603887.SH     5.2400
 603890.SH    17.9800
 603900.SH     5.5100
 603901.SH     4.6750
 603903.SH     5.3200
 603906.SH     2.8000
 603908.SH     2.1100
 603909.SH    14.3580
 603917.SH     1.2800
 603918.SH    1

## 十大流通股东信托占比
#### 信托公司的公司名称都包含哪些我不太清楚，这里就只写了公司名称里面包含信托的

In [186]:
def get_trust_holder_info(date,conn):
    sql="""select S_INFO_WINDCODE,S_HOLDER_NAME,S_HOLDER_QUANTITY,REPORT_PERIOD
            from ASHAREFLOATHOLDER where ANN_DT<='{}' and REPORT_PERIOD>='{}' and S_HOLDER_HOLDERCATEGORY='2'"""\
            .format(get_next_trade_date(date),get_prev_n_trade_date(date,150))

    df_float_holder=pd.read_sql_query(sql,conn)
    
    ###获得最新报告期的前十大流通股东信息
    def get_latest_top10(df):
        latest_repper=df["REPORT_PERIOD"][df.index[-1]]
        df=df[df["REPORT_PERIOD"]==latest_repper]
        df=df.sort_values("S_HOLDER_QUANTITY",ascending=False).iloc[0:10]
        return df
    df_float_top10_holder=df_float_holder.sort_values(['S_INFO_WINDCODE','REPORT_PERIOD'])\
                                   .groupby('S_INFO_WINDCODE').apply(get_latest_top10)
    ###提取出信托持股信息
    def get_trust_info(df):
        df["is_trust"]=df["S_HOLDER_NAME"].apply(lambda x:'信托' in x)
        trust_ratio,trust_num=(np.sum(df["S_HOLDER_QUANTITY"]*df["is_trust"])/np.sum(df["S_HOLDER_QUANTITY"]),np.sum(df["is_trust"]))
        return pd.Series([trust_ratio,trust_num],index=["trust_ratio","trust_num"])
    df_float_top10_trust_info=df_float_top10_holder.groupby("S_INFO_WINDCODE").apply(get_trust_info)
    return df_float_top10_trust_info
get_trust_holder_info(date,conn)

/home/ywang/env/dev/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: 'S_INFO_WINDCODE' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version


,trust_ratio,trust_num
S_INFO_WINDCODE,,
000001.SZ,0.000000,0.0
000002.SZ,0.000000,0.0
000004.SZ,0.000000,0.0
000005.SZ,0.000000,0.0
000006.SZ,0.000000,0.0
000007.SZ,0.000000,0.0
000008.SZ,0.028106,1.0
000009.SZ,0.000000,0.0
000010.SZ,0.000000,0.0


## CFO离职、高管离职

In [187]:
def get_manager_leave_info(date,conn):    
    sql="""select S_INFO_WINDCODE,S_INFO_MANAGER_POST,S_INFO_MANAGER_LEAVEDATE
            from ASHAREMANAGEMENT where ANN_DATE<='{}' and S_INFO_MANAGER_LEAVEDATE>='{}' and S_INFO_MANAGER_LEAVEDATE<='{}'"""\
            .format(date,get_prev_n_trade_date(date,63),date)
    df_manager_info=pd.read_sql_query(sql,conn)


    def get_manager_leave_info(df):
        df["is_cfo_leave"]=df["S_INFO_MANAGER_POST"].apply(lambda x:x in ["财务总监","财务负责人","总会计师","首席财务官","代财务总监"])
        cfo_leave=(np.sum(df["is_cfo_leave"])>0)

        manager_leave_num=len(df.index)
        manager_leave_biggerthan3=(manager_leave_num>3)

        return pd.Series([cfo_leave,manager_leave_biggerthan3],index=["cfo_leave","manager_leave_num>3"])
    df_manager_leave=df_manager_info.groupby("S_INFO_WINDCODE").apply(get_manager_leave_info)
    return df_manager_leave
get_manager_leave_info(date,conn)

,cfo_leave,manager_leave_num>3
S_INFO_WINDCODE,,
000001.SZ,False,True
000004.SZ,False,True
000007.SZ,True,True
000015.SZ,False,False
000018.SZ,True,True
000019.SZ,False,False
000020.SZ,False,True
000023.SZ,True,True
000026.SZ,False,False


## 财报披露调整

In [188]:
def get_report_adj_overdue_info(date,conn):    
    sql="""select S_INFO_WINDCODE,REPORT_PERIOD,S_STM_PREDICT_ISSUINGDATE,S_STM_ACTUAL_ISSUINGDATE,ANN_DT
            from ASHAREISSUINGDATEPREDICT where ANN_DT<='{}' and ANN_DT>='{}'"""\
            .format(date,get_prev_n_trade_date(date,252))
    df_reportdate_adj=pd.read_sql_query(sql,conn)
    
    def get_due_date(rep_per):
        if rep_per[4:8]=='1231':
            return str(int(rep_per[0:4])+1)+"0430"
        elif rep_per[4:8]=='0331':
            return rep_per[0:4]+"0430"
        elif rep_per[4:8]=='0630':
            return rep_per[0:4]+"0830"
        elif rep_per[4:8]=='0930':
            return rep_per[0:4]+"1031"
        
    def rep_date_adj_and_overdue(df):
        rep_per=df["REPORT_PERIOD"][df.index[0]]
        due_date=get_due_date(rep_per)
        is_overdue=(df["S_STM_ACTUAL_ISSUINGDATE"]>due_date).sum()
        is_adj=(len(df.index)>1)

        return pd.Series([is_adj,is_overdue],index=["is_adj","is_overdue"])
    df_reportdate_adj=df_reportdate_adj.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD","S_STM_PREDICT_ISSUINGDATE"])\
                                       .groupby(["S_INFO_WINDCODE","REPORT_PERIOD"]).apply(rep_date_adj_and_overdue)
    df_reportdate_adj=df_reportdate_adj.groupby("S_INFO_WINDCODE").apply(lambda x:(np.sum(x,axis=0)>0))
    return df_reportdate_adj
get_report_adj_overdue_info(date,conn)

,is_adj,is_overdue
S_INFO_WINDCODE,,
000001.SZ,False,False
000002.SZ,False,False
000004.SZ,False,True
000005.SZ,False,False
000006.SZ,False,False
000007.SZ,False,True
000008.SZ,False,False
000009.SZ,False,True
000010.SZ,False,False


## 违规、立案调查、诉讼

In [189]:
def get_illegal_reginv_prosecute_info(date,conn):
    sql_1="""select S_INFO_WINDCODE,ANN_DT
            from AShareIllegality where ANN_DT<='{}' and ANN_DT>='{}'"""\
            .format(date,get_prev_n_trade_date(date,252)).upper()
    df_illegality=pd.read_sql_query(sql_1,conn)
    df_illegality=df_illegality.groupby("S_INFO_WINDCODE").apply(lambda x:(len(x)>0))

    sql_2="""select S_INFO_WINDCODE,STR_DATE
            from AShareRegInv where STR_DATE<='{}' and STR_DATE>='{}'"""\
            .format(date,get_prev_n_trade_date(date,252)).upper()
    df_reginv=pd.read_sql_query(sql_2,conn)
    df_reginv=df_reginv.groupby("S_INFO_WINDCODE").apply(lambda x:(len(x)>0))

    sql_3="""select S_INFO_WINDCODE,ANN_DT
            from AShareProsecution where ANN_DT<='{}' and ANN_DT>='{}'"""\
            .format(date,get_prev_n_trade_date(date,252)).upper()
    df_prosecution=pd.read_sql_query(sql_3,conn)
    df_prosecution=df_prosecution.groupby("S_INFO_WINDCODE").apply(lambda x:(len(x)>3))

    return df_illegality,df_reginv,df_prosecution
get_illegal_reginv_prosecute_info(date,conn)

(S_INFO_WINDCODE
 000001.SZ    True
 000004.SZ    True
 000007.SZ    True
 000010.SZ    True
 000012.SZ    True
 000018.SZ    True
 000023.SZ    True
 000038.SZ    True
 000042.SZ    True
 000046.SZ    True
 000048.SZ    True
 000065.SZ    True
 000150.SZ    True
 000333.SZ    True
 000403.SZ    True
 000407.SZ    True
 000408.SZ    True
 000409.SZ    True
 000410.SZ    True
 000415.SZ    True
 000422.SZ    True
 000426.SZ    True
 000488.SZ    True
 000501.SZ    True
 000506.SZ    True
 000509.SZ    True
 000518.SZ    True
 000519.SZ    True
 000529.SZ    True
 000534.SZ    True
              ... 
 603955.SH    True
 603997.SH    True
 603998.SH    True
 688005.SH    True
 688388.SH    True
 A11202.SZ    True
 A11230.SZ    True
 A11242.SZ    True
 A12289.SZ    True
 A15130.SZ    True
 A15163.SH    True
 A15278.SH    True
 A15295.SZ    True
 A16059.SH    True
 A16061.SH    True
 A16230.SZ    True
 A16268.SZ    True
 A16278.SH    True
 A16319.SH    True
 A17217.SH    True
 A17242.SH    

## 业绩预告不准确度

In [190]:
def get_unaccuracy_notice(date,conn):    
    sql = """SELECT S_INFO_WINDCODE, S_PROFITNOTICE_PERIOD,S_PROFITNOTICE_NETPROFITMIN
            from ASHAREPROFITNOTICE where S_PROFITNOTICE_DATE<='{}' and S_PROFITNOTICE_DATE>='{}'"""\
            .format(date,get_prev_n_trade_date(date,350))

    df_notice_profit_lowerbound = pd.read_sql_query(sql, conn)
    df_notice_profit_lowerbound=df_notice_profit_lowerbound.sort_values(["S_INFO_WINDCODE","S_PROFITNOTICE_PERIOD"])\
                                                           .groupby("S_INFO_WINDCODE").apply(lambda x:x.iloc[-4:])\
                                                           .rename(columns={"S_PROFITNOTICE_PERIOD":"REPORT_PERIOD"})
    sql="""select S_INFO_WINDCODE,REPORT_PERIOD,NET_PROFIT_INCL_MIN_INT_INC from ASHAREINCOME 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and ANN_DT>='{}'"""\
        .format(get_next_trade_date(date),get_prev_n_trade_date(date,350))
    df_real_profit=pd.read_sql_query(sql,conn)
    df_real_profit=df_real_profit.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"])\
                                  .groupby("S_INFO_WINDCODE").apply(lambda x:x.iloc[-4:])\


    df_notice_real_profit=pd.merge(df_notice_profit_lowerbound,df_real_profit,on=["S_INFO_WINDCODE","REPORT_PERIOD"],how="left")
    df_unaccuracy_notice=df_notice_real_profit.groupby("S_INFO_WINDCODE")\
                                              .apply(lambda x:(x["NET_PROFIT_INCL_MIN_INT_INC"]<x["S_PROFITNOTICE_NETPROFITMIN"]).sum()>0)
    return df_unaccuracy_notice
get_unaccuracy_notice(date,conn)

S_INFO_WINDCODE
000004.SZ     True
000005.SZ     True
000006.SZ    False
000007.SZ     True
000008.SZ    False
000009.SZ    False
000010.SZ     True
000011.SZ    False
000014.SZ     True
000016.SZ    False
000017.SZ     True
000018.SZ     True
000019.SZ    False
000023.SZ    False
000025.SZ    False
000026.SZ    False
000027.SZ    False
000029.SZ    False
000031.SZ    False
000032.SZ    False
000034.SZ    False
000035.SZ    False
000036.SZ    False
000037.SZ     True
000038.SZ     True
000039.SZ    False
000040.SZ    False
000042.SZ    False
000043.SZ    False
000045.SZ     True
             ...  
688088.SH    False
688098.SH    False
688099.SH    False
688101.SH    False
688108.SH    False
688111.SH    False
688122.SH    False
688128.SH    False
688138.SH    False
688139.SH    False
688166.SH    False
688188.SH    False
688196.SH    False
688198.SH    False
688199.SH    False
688218.SH    False
688258.SH    False
688288.SH    False
688299.SH    False
688300.SH    False
688310.SH    Fa

## 商誉

In [191]:
def get_goodwill_info(date,conn):    
    sql=(f"""select S_INFO_WINDCODE, ANN_DT, REPORT_PERIOD, TOT_ASSETS, TOT_LIAB,GOODWILL from ASHAREBALANCESHEET"""
            + f""" where STATEMENT_TYPE='408001000' and REPORT_PERIOD>='%s' and ANN_DT<='%s'""" % (get_prev_n_trade_date(date,150), get_next_trade_date(date)))
    df_goodwill = pd.read_sql_query(sql,conn)

    df_goodwill=df_goodwill.sort_values(["S_INFO_WINDCODE","REPORT_PERIOD"]).groupby("S_INFO_WINDCODE").apply(lambda x:x.iloc[-1])
    df_goodwill_ratio=df_goodwill["GOODWILL"]/(df_goodwill["TOT_ASSETS"]-df_goodwill["TOT_LIAB"])
    return df_goodwill_ratio
get_goodwill_info(date,conn)

S_INFO_WINDCODE
000001.SZ    0.026271
000002.SZ    0.000921
000003.SZ         NaN
000004.SZ         NaN
000005.SZ    0.136496
000006.SZ         NaN
000007.SZ         NaN
000008.SZ    0.492519
000009.SZ    0.047620
000010.SZ    0.281133
000011.SZ         NaN
000012.SZ    0.038011
000014.SZ         NaN
000016.SZ    0.084785
000017.SZ         NaN
000018.SZ   -0.021521
000019.SZ         NaN
000020.SZ         NaN
000021.SZ    0.001558
000023.SZ         NaN
000025.SZ         NaN
000026.SZ         NaN
000027.SZ    0.076375
000028.SZ    0.058389
000029.SZ         NaN
000030.SZ    0.001449
000031.SZ    0.008518
000032.SZ         NaN
000034.SZ    0.221891
000035.SZ    0.556804
               ...   
A19411.SH         NaN
A19413.SH    0.174036
A19418.SH         NaN
A19423.SH    0.024583
A19425.SZ    0.015128
A19427.SH    0.015178
A19428.SH         NaN
A19429.SH         NaN
A19430.SZ         NaN
A19431.SH         NaN
A19432.SH         NaN
A19433.SH         NaN
A19434.SH         NaN
A19435.SH    0.0

## 并购次数

In [192]:
def get_merge_times_info(date,conn):
    sql="""select EVENT_ID,ANN_DATE from MERGEREVENT where ANN_DATE>={} and ANN_DATE<={}""".format(get_prev_n_trade_date(date,750), date)
    df_merge_event=pd.read_sql(sql,conn)
    sql="""select EVENT_ID,S_INFO_WINDCODE,RELATIONSHIP from MERGERPARTICIPANT"""
    df_merge_participant=pd.read_sql(sql,conn)

    df_merge_participant=df_merge_participant[df_merge_participant["RELATIONSHIP"]==323001000]#上市公司本身参与并购
    df_merge_participant=df_merge_participant[pd.notnull(df_merge_participant["S_INFO_WINDCODE"])]
    df_merge_event=pd.merge(df_merge_event,df_merge_participant,on="EVENT_ID",how="right")

    df_merge_times=df_merge_event.groupby("S_INFO_WINDCODE").apply(lambda x:len(x))
    return df_merge_times
get_merge_times_info(date,conn)

S_INFO_WINDCODE
000001.SZ      8
000004.SZ     21
000007.SZ      4
000008.SZ      3
000009.SZ     14
000010.SZ     17
000011.SZ     11
000015.SZ      2
000016.SZ     20
000018.SZ      3
000023.SZ      8
000025.SZ      4
000027.SZ      8
000028.SZ      7
000030.KS      2
000030.SZ      3
000031.SZ     13
000032.SZ      1
000034.SZ      5
000035.SZ      4
000036.SZ      1
000037.SZ      6
000038.SZ      5
000039.SZ     12
000040.SZ      7
000042.SZ      2
000043.SZ     18
000045.SZ      8
000046.SZ     10
000048.SZ     10
              ..
XTC.TO         4
XTLY.N         2
XTV.AX         2
YAL.AX         4
YBR.AX         8
YEW.IP         4
YGYI.O        10
YNDX.O         6
YOU.L          8
YPF.N          2
YTRA.O         2
YUMC.N         1
YVR.O          2
Z.O            2
ZAGG.O         4
ZBRA.O         4
ZCL.TO         2
ZEAL.CO        4
ZEL.NZ         2
ZETA.ST        6
ZEUS.O         2
ZG.O           2
ZGNX.O         2
ZIP.AX         2
ZIXI.O        14
ZLAB.O         2
ZNGA.O         

## Mscore 打分

In [ ]:
def get_Mscore_info(date,conn):
    sql_1="""select S_INFO_WINDCODE,REPORT_PERIOD,ACCT_RCV,FIX_ASSETS from ASHAREBALANCESHEET 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-2)

    sql_2="""select S_INFO_WINDCODE,REPORT_PERIOD,OPER_PROFIT,OPER_REV from ASHAREINCOME 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-2)

    sql_3="""select S_INFO_WINDCODE,REPORT_PERIOD,NET_CASH_FLOWS_OPER_ACT from ASHARECASHFLOW 
        where STATEMENT_TYPE ='408001000'and ANN_DT<='{}' and REPORT_PERIOD>='{}1231' and REPORT_PERIOD like '%1231'"""\
        .format(get_next_trade_date(date),int(date[0:4])-2)
